# Lab 3.6.3: LangGraph Workflow - SOLUTIONS

**Complete solutions with explanations and alternative approaches**

---

## Setup

In [ ]:
from typing import Dict, Any, List, Optional, TypedDict, Annotated, Literal
from dataclasses import dataclass, field
from enum import Enum
import operator
import json
from datetime import datetime

print("Setup complete!")

---

## Exercise 1 Solution: Document Processing Pipeline

**Task**: Build a multi-stage document processing workflow with branching logic.

In [ ]:
class DocumentState(TypedDict):
    """State for document processing workflow."""
    document: str
    document_type: str
    extracted_entities: List[str]
    summary: str
    sentiment: str
    keywords: List[str]
    metadata: Dict[str, Any]
    processing_steps: List[str]
    errors: List[str]


class DocumentWorkflow:
    """
    Multi-stage document processing workflow.
    
    Pipeline:
    1. Classify document type
    2. Extract entities (conditional on type)
    3. Generate summary
    4. Analyze sentiment
    5. Extract keywords
    6. Compile results
    """
    
    def __init__(self, verbose: bool = True):
        self.verbose = verbose
    
    def _log(self, step: str, message: str):
        if self.verbose:
            print(f"  [{step}] {message}")
    
    def classify_document(self, state: DocumentState) -> DocumentState:
        """Classify document type."""
        doc = state["document"].lower()
        
        # Simple classification rules
        if any(word in doc for word in ["invoice", "total", "payment", "due"]):
            doc_type = "invoice"
        elif any(word in doc for word in ["dear", "sincerely", "regards"]):
            doc_type = "letter"
        elif any(word in doc for word in ["whereas", "hereby", "agreement", "contract"]):
            doc_type = "legal"
        elif any(word in doc for word in ["abstract", "methodology", "results", "conclusion"]):
            doc_type = "research"
        else:
            doc_type = "general"
        
        state["document_type"] = doc_type
        state["processing_steps"].append(f"classified as {doc_type}")
        self._log("Classify", f"Document type: {doc_type}")
        
        return state
    
    def extract_entities(self, state: DocumentState) -> DocumentState:
        """Extract entities based on document type."""
        doc = state["document"]
        entities = []
        
        # Type-specific extraction
        if state["document_type"] == "invoice":
            # Extract amounts
            import re
            amounts = re.findall(r'\$[\d,]+\.?\d*', doc)
            entities.extend([f"amount: {a}" for a in amounts])
            
            # Extract dates
            dates = re.findall(r'\d{1,2}/\d{1,2}/\d{2,4}', doc)
            entities.extend([f"date: {d}" for d in dates])
        
        elif state["document_type"] == "legal":
            # Extract party names (simplified)
            import re
            parties = re.findall(r'"([^"]+)"', doc)
            entities.extend([f"party: {p}" for p in parties])
        
        else:
            # General entity extraction (capitalized words)
            import re
            names = re.findall(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)+\b', doc)
            entities.extend([f"name: {n}" for n in names[:5]])
        
        state["extracted_entities"] = entities
        state["processing_steps"].append(f"extracted {len(entities)} entities")
        self._log("Extract", f"Found {len(entities)} entities")
        
        return state
    
    def generate_summary(self, state: DocumentState) -> DocumentState:
        """Generate document summary."""
        doc = state["document"]
        
        # Simple extractive summary (first 2 sentences)
        sentences = doc.replace('!', '.').replace('?', '.').split('.')
        sentences = [s.strip() for s in sentences if s.strip()]
        
        summary = '. '.join(sentences[:2]) + '.' if sentences else "No summary available."
        
        state["summary"] = summary
        state["processing_steps"].append("generated summary")
        self._log("Summarize", f"Summary: {summary[:50]}...")
        
        return state
    
    def analyze_sentiment(self, state: DocumentState) -> DocumentState:
        """Analyze document sentiment."""
        doc = state["document"].lower()
        
        # Simple sentiment analysis
        positive_words = ['good', 'great', 'excellent', 'happy', 'pleased', 'thank', 'appreciate']
        negative_words = ['bad', 'poor', 'issue', 'problem', 'complaint', 'disappointed', 'sorry']
        
        pos_count = sum(1 for word in positive_words if word in doc)
        neg_count = sum(1 for word in negative_words if word in doc)
        
        if pos_count > neg_count:
            sentiment = "positive"
        elif neg_count > pos_count:
            sentiment = "negative"
        else:
            sentiment = "neutral"
        
        state["sentiment"] = sentiment
        state["processing_steps"].append(f"sentiment: {sentiment}")
        self._log("Sentiment", f"Detected: {sentiment}")
        
        return state
    
    def extract_keywords(self, state: DocumentState) -> DocumentState:
        """Extract keywords from document."""
        import re
        from collections import Counter
        
        doc = state["document"].lower()
        
        # Remove common words
        stop_words = {'the', 'a', 'an', 'is', 'are', 'was', 'were', 'be', 'been', 
                      'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will',
                      'would', 'could', 'should', 'may', 'might', 'must', 'shall',
                      'to', 'of', 'in', 'for', 'on', 'with', 'at', 'by', 'from',
                      'as', 'into', 'through', 'during', 'before', 'after', 'and',
                      'but', 'or', 'nor', 'so', 'yet', 'both', 'either', 'neither',
                      'not', 'only', 'own', 'same', 'than', 'too', 'very', 'just',
                      'this', 'that', 'these', 'those', 'i', 'you', 'he', 'she', 'it',
                      'we', 'they', 'what', 'which', 'who', 'whom', 'whose', 'where',
                      'when', 'why', 'how'}
        
        # Extract words
        words = re.findall(r'\b[a-z]+\b', doc)
        words = [w for w in words if w not in stop_words and len(w) > 3]
        
        # Get top keywords
        word_counts = Counter(words)
        keywords = [word for word, count in word_counts.most_common(10)]
        
        state["keywords"] = keywords
        state["processing_steps"].append(f"extracted {len(keywords)} keywords")
        self._log("Keywords", f"Top keywords: {keywords[:5]}")
        
        return state
    
    def compile_results(self, state: DocumentState) -> DocumentState:
        """Compile final results with metadata."""
        state["metadata"] = {
            "processed_at": datetime.now().isoformat(),
            "document_length": len(state["document"]),
            "word_count": len(state["document"].split()),
            "steps_completed": len(state["processing_steps"]),
        }
        
        state["processing_steps"].append("compilation complete")
        self._log("Compile", "Results compiled")
        
        return state
    
    def process(self, document: str) -> DocumentState:
        """Run full processing pipeline."""
        # Initialize state
        state: DocumentState = {
            "document": document,
            "document_type": "",
            "extracted_entities": [],
            "summary": "",
            "sentiment": "",
            "keywords": [],
            "metadata": {},
            "processing_steps": [],
            "errors": [],
        }
        
        print("\n" + "="*60)
        print("DOCUMENT PROCESSING PIPELINE")
        print("="*60)
        
        # Run pipeline
        pipeline = [
            self.classify_document,
            self.extract_entities,
            self.generate_summary,
            self.analyze_sentiment,
            self.extract_keywords,
            self.compile_results,
        ]
        
        for step_fn in pipeline:
            try:
                state = step_fn(state)
            except Exception as e:
                state["errors"].append(f"{step_fn.__name__}: {str(e)}")
                self._log("ERROR", str(e))
        
        return state


# Test the workflow
workflow = DocumentWorkflow(verbose=True)

test_documents = [
    """Invoice #12345
    Date: 01/15/2024
    
    Services rendered: Consulting
    Amount due: $5,000.00
    Payment due by: 02/15/2024
    
    Thank you for your business!""",
    
    """Dear John Smith,
    
    I am pleased to inform you that your application has been approved.
    We are excited to have you join our team at Acme Corporation.
    
    Sincerely,
    Jane Doe
    HR Manager""",
]

for doc in test_documents:
    result = workflow.process(doc)
    print(f"\n--- Results ---")
    print(f"Type: {result['document_type']}")
    print(f"Sentiment: {result['sentiment']}")
    print(f"Keywords: {result['keywords'][:5]}")
    print(f"Entities: {result['extracted_entities']}")
    print(f"Summary: {result['summary'][:100]}...")

---

## Exercise 2 Solution: Approval Workflow with Human-in-the-Loop

**Task**: Build a workflow that pauses for human approval at critical decision points.

In [ ]:
class ApprovalStatus(Enum):
    PENDING = "pending"
    APPROVED = "approved"
    REJECTED = "rejected"
    AUTO_APPROVED = "auto_approved"


class ApprovalWorkflowState(TypedDict):
    """State for approval workflow."""
    request_id: str
    request_type: str
    amount: float
    requester: str
    description: str
    risk_score: float
    approval_status: str
    approver: Optional[str]
    approval_notes: str
    history: List[str]


class ApprovalWorkflow:
    """
    Approval workflow with human-in-the-loop.
    
    Flow:
    1. Validate request
    2. Calculate risk score
    3. Route based on amount/risk:
       - Low: Auto-approve
       - Medium: Single approver
       - High: Manager approval required
    4. Process approval decision
    5. Execute action
    """
    
    def __init__(self, auto_approve_limit: float = 100.0, 
                 manager_threshold: float = 1000.0):
        self.auto_approve_limit = auto_approve_limit
        self.manager_threshold = manager_threshold
        self.pending_approvals: Dict[str, ApprovalWorkflowState] = {}
    
    def validate_request(self, state: ApprovalWorkflowState) -> ApprovalWorkflowState:
        """Validate the request."""
        errors = []
        
        if state["amount"] <= 0:
            errors.append("Amount must be positive")
        if not state["requester"]:
            errors.append("Requester is required")
        if not state["description"]:
            errors.append("Description is required")
        
        if errors:
            state["approval_status"] = ApprovalStatus.REJECTED.value
            state["approval_notes"] = f"Validation failed: {', '.join(errors)}"
        
        state["history"].append(f"Validated: {'passed' if not errors else 'failed'}")
        return state
    
    def calculate_risk(self, state: ApprovalWorkflowState) -> ApprovalWorkflowState:
        """Calculate risk score for the request."""
        risk = 0.0
        
        # Amount-based risk
        if state["amount"] > 5000:
            risk += 0.4
        elif state["amount"] > 1000:
            risk += 0.2
        elif state["amount"] > 500:
            risk += 0.1
        
        # Type-based risk
        high_risk_types = ["external_payment", "new_vendor", "emergency"]
        if state["request_type"] in high_risk_types:
            risk += 0.3
        
        # Description keywords
        desc_lower = state["description"].lower()
        if any(word in desc_lower for word in ["urgent", "immediate", "asap"]):
            risk += 0.2
        
        state["risk_score"] = min(1.0, risk)
        state["history"].append(f"Risk calculated: {state['risk_score']:.2f}")
        return state
    
    def route_request(self, state: ApprovalWorkflowState) -> str:
        """Determine routing based on amount and risk."""
        if state["approval_status"] == ApprovalStatus.REJECTED.value:
            return "rejected"
        
        # Auto-approve low-risk, low-amount
        if state["amount"] <= self.auto_approve_limit and state["risk_score"] < 0.2:
            return "auto_approve"
        
        # Manager approval for high amount or risk
        if state["amount"] > self.manager_threshold or state["risk_score"] > 0.5:
            return "manager_approval"
        
        return "standard_approval"
    
    def auto_approve(self, state: ApprovalWorkflowState) -> ApprovalWorkflowState:
        """Auto-approve low-risk requests."""
        state["approval_status"] = ApprovalStatus.AUTO_APPROVED.value
        state["approver"] = "SYSTEM"
        state["approval_notes"] = "Auto-approved: Low risk, within limits"
        state["history"].append("Auto-approved by system")
        return state
    
    def request_human_approval(self, state: ApprovalWorkflowState, 
                               approval_type: str) -> ApprovalWorkflowState:
        """Mark request as pending human approval."""
        state["approval_status"] = ApprovalStatus.PENDING.value
        state["approval_notes"] = f"Awaiting {approval_type}"
        state["history"].append(f"Pending {approval_type}")
        
        # Store for later approval
        self.pending_approvals[state["request_id"]] = state
        
        return state
    
    def process_approval(self, request_id: str, approved: bool, 
                        approver: str, notes: str = "") -> ApprovalWorkflowState:
        """Process human approval decision."""
        if request_id not in self.pending_approvals:
            raise ValueError(f"No pending approval for {request_id}")
        
        state = self.pending_approvals.pop(request_id)
        
        state["approver"] = approver
        state["approval_notes"] = notes
        
        if approved:
            state["approval_status"] = ApprovalStatus.APPROVED.value
            state["history"].append(f"Approved by {approver}")
        else:
            state["approval_status"] = ApprovalStatus.REJECTED.value
            state["history"].append(f"Rejected by {approver}: {notes}")
        
        return state
    
    def execute(self, state: ApprovalWorkflowState) -> ApprovalWorkflowState:
        """Execute approved action."""
        if state["approval_status"] in [ApprovalStatus.APPROVED.value, 
                                         ApprovalStatus.AUTO_APPROVED.value]:
            state["history"].append("Action executed successfully")
        else:
            state["history"].append("No action taken - not approved")
        
        return state
    
    def run(self, request_id: str, request_type: str, amount: float,
            requester: str, description: str) -> ApprovalWorkflowState:
        """Run the approval workflow."""
        # Initialize state
        state: ApprovalWorkflowState = {
            "request_id": request_id,
            "request_type": request_type,
            "amount": amount,
            "requester": requester,
            "description": description,
            "risk_score": 0.0,
            "approval_status": "",
            "approver": None,
            "approval_notes": "",
            "history": [],
        }
        
        print(f"\n--- Processing Request {request_id} ---")
        print(f"Type: {request_type}, Amount: ${amount:.2f}, Requester: {requester}")
        
        # Run workflow steps
        state = self.validate_request(state)
        state = self.calculate_risk(state)
        
        # Route based on risk
        route = self.route_request(state)
        print(f"Routing: {route}")
        
        if route == "auto_approve":
            state = self.auto_approve(state)
            state = self.execute(state)
        elif route == "standard_approval":
            state = self.request_human_approval(state, "standard approval")
        elif route == "manager_approval":
            state = self.request_human_approval(state, "manager approval")
        
        print(f"Status: {state['approval_status']}")
        
        return state


# Test the approval workflow
print("=" * 60)
print("APPROVAL WORKFLOW SOLUTION")
print("=" * 60)

workflow = ApprovalWorkflow(auto_approve_limit=100, manager_threshold=1000)

# Test cases
requests = [
    ("REQ001", "expense", 50.0, "alice", "Office supplies"),
    ("REQ002", "expense", 500.0, "bob", "Team dinner"),
    ("REQ003", "external_payment", 2000.0, "charlie", "URGENT vendor payment"),
]

for req in requests:
    result = workflow.run(*req)
    print(f"History: {result['history']}\n")

# Simulate human approval
print("\n--- Simulating Human Approval ---")
if workflow.pending_approvals:
    for req_id in list(workflow.pending_approvals.keys()):
        state = workflow.process_approval(
            req_id, 
            approved=True, 
            approver="manager_jane",
            notes="Approved after review"
        )
        state = workflow.execute(state)
        print(f"{req_id}: {state['approval_status']} by {state['approver']}")

---

## Exercise 3 Solution: Parallel Research Workflow

**Task**: Build a workflow that executes multiple research tasks in parallel and aggregates results.

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random


class ResearchState(TypedDict):
    """State for research workflow."""
    query: str
    sources: List[str]
    results: Dict[str, Any]
    aggregated: str
    confidence: float
    metadata: Dict[str, Any]


class ParallelResearchWorkflow:
    """
    Parallel research workflow that queries multiple sources.
    
    Flow:
    1. Parse query and identify sources
    2. Execute parallel queries
    3. Aggregate results
    4. Generate final answer
    """
    
    def __init__(self, max_workers: int = 4):
        self.max_workers = max_workers
        self.sources = {
            "wikipedia": self._search_wikipedia,
            "news": self._search_news,
            "academic": self._search_academic,
            "web": self._search_web,
        }
    
    def _search_wikipedia(self, query: str) -> Dict[str, Any]:
        """Simulated Wikipedia search."""
        time.sleep(random.uniform(0.1, 0.3))  # Simulate network delay
        return {
            "source": "wikipedia",
            "content": f"Wikipedia article about {query}. Contains factual information.",
            "reliability": 0.85,
            "citations": 15,
        }
    
    def _search_news(self, query: str) -> Dict[str, Any]:
        """Simulated news search."""
        time.sleep(random.uniform(0.1, 0.3))
        return {
            "source": "news",
            "content": f"Recent news articles about {query}. Current events coverage.",
            "reliability": 0.7,
            "articles": 8,
        }
    
    def _search_academic(self, query: str) -> Dict[str, Any]:
        """Simulated academic search."""
        time.sleep(random.uniform(0.2, 0.4))
        return {
            "source": "academic",
            "content": f"Academic papers on {query}. Peer-reviewed research.",
            "reliability": 0.95,
            "papers": 5,
        }
    
    def _search_web(self, query: str) -> Dict[str, Any]:
        """Simulated web search."""
        time.sleep(random.uniform(0.1, 0.2))
        return {
            "source": "web",
            "content": f"General web results for {query}. Mixed quality.",
            "reliability": 0.5,
            "results": 100,
        }
    
    def parallel_search(self, query: str, sources: List[str]) -> Dict[str, Any]:
        """Execute searches in parallel."""
        results = {}
        
        with ThreadPoolExecutor(max_workers=self.max_workers) as executor:
            # Submit all tasks
            future_to_source = {
                executor.submit(self.sources[src], query): src
                for src in sources if src in self.sources
            }
            
            # Collect results as they complete
            for future in as_completed(future_to_source):
                source = future_to_source[future]
                try:
                    results[source] = future.result()
                except Exception as e:
                    results[source] = {"error": str(e)}
        
        return results
    
    def aggregate_results(self, results: Dict[str, Any]) -> tuple:
        """Aggregate results from multiple sources."""
        # Collect content weighted by reliability
        weighted_content = []
        total_reliability = 0
        
        for source, data in results.items():
            if "error" not in data:
                reliability = data.get("reliability", 0.5)
                weighted_content.append((data["content"], reliability))
                total_reliability += reliability
        
        # Create aggregated summary
        if not weighted_content:
            return "No results found", 0.0
        
        # Sort by reliability
        weighted_content.sort(key=lambda x: x[1], reverse=True)
        
        # Combine top sources
        aggregated = "\n".join([c[0] for c in weighted_content[:3]])
        confidence = total_reliability / len(results) if results else 0
        
        return aggregated, confidence
    
    def run(self, query: str, sources: Optional[List[str]] = None) -> ResearchState:
        """Run the research workflow."""
        if sources is None:
            sources = list(self.sources.keys())
        
        print(f"\n--- Research Query: {query} ---")
        print(f"Sources: {sources}")
        
        start_time = time.time()
        
        # Parallel search
        results = self.parallel_search(query, sources)
        search_time = time.time() - start_time
        
        # Aggregate
        aggregated, confidence = self.aggregate_results(results)
        
        state: ResearchState = {
            "query": query,
            "sources": sources,
            "results": results,
            "aggregated": aggregated,
            "confidence": confidence,
            "metadata": {
                "search_time_ms": search_time * 1000,
                "sources_queried": len(sources),
                "successful_sources": len([r for r in results.values() if "error" not in r]),
            },
        }
        
        print(f"Completed in {search_time*1000:.1f}ms")
        print(f"Confidence: {confidence:.2f}")
        
        return state


# Test parallel research
print("=" * 60)
print("PARALLEL RESEARCH WORKFLOW SOLUTION")
print("=" * 60)

research = ParallelResearchWorkflow(max_workers=4)

queries = [
    "artificial intelligence in healthcare",
    "climate change solutions",
]

for query in queries:
    result = research.run(query)
    print(f"\nAggregated Answer:")
    print(result["aggregated"][:200] + "...")
    print(f"\nMetadata: {result['metadata']}")

---

## Challenge Solution: Conditional Workflow Builder

**Task**: Create a DSL (Domain Specific Language) for building workflows declaratively.

In [ ]:
from typing import Callable, Union


class WorkflowNode:
    """A node in the workflow graph."""
    
    def __init__(self, name: str, handler: Callable):
        self.name = name
        self.handler = handler
        self.next_nodes: List[tuple] = []  # (condition, node_name)
    
    def then(self, next_node: Union[str, 'WorkflowNode']) -> 'WorkflowNode':
        """Add unconditional transition."""
        node_name = next_node if isinstance(next_node, str) else next_node.name
        self.next_nodes.append((None, node_name))
        return self
    
    def when(self, condition: Callable, next_node: Union[str, 'WorkflowNode']) -> 'WorkflowNode':
        """Add conditional transition."""
        node_name = next_node if isinstance(next_node, str) else next_node.name
        self.next_nodes.append((condition, node_name))
        return self


class WorkflowBuilder:
    """
    Declarative workflow builder.
    
    Usage:
        builder = WorkflowBuilder()
        builder.add_node("start", start_handler)
        builder.add_node("process", process_handler)
        builder.add_node("end", end_handler)
        
        builder.connect("start", "process")
        builder.connect("process", "end", condition=lambda s: s["status"] == "ok")
        
        workflow = builder.build()
    """
    
    def __init__(self):
        self.nodes: Dict[str, WorkflowNode] = {}
        self.start_node: Optional[str] = None
        self.end_nodes: List[str] = []
    
    def add_node(self, name: str, handler: Callable) -> WorkflowNode:
        """Add a node to the workflow."""
        node = WorkflowNode(name, handler)
        self.nodes[name] = node
        return node
    
    def set_start(self, name: str) -> 'WorkflowBuilder':
        """Set the starting node."""
        self.start_node = name
        return self
    
    def set_end(self, *names: str) -> 'WorkflowBuilder':
        """Set end node(s)."""
        self.end_nodes.extend(names)
        return self
    
    def connect(self, from_node: str, to_node: str, 
                condition: Optional[Callable] = None) -> 'WorkflowBuilder':
        """Connect two nodes."""
        if from_node not in self.nodes:
            raise ValueError(f"Node not found: {from_node}")
        
        self.nodes[from_node].next_nodes.append((condition, to_node))
        return self
    
    def build(self) -> 'Workflow':
        """Build the workflow."""
        if not self.start_node:
            raise ValueError("Start node not set")
        
        return Workflow(self.nodes, self.start_node, self.end_nodes)


class Workflow:
    """Executable workflow."""
    
    def __init__(self, nodes: Dict[str, WorkflowNode], 
                 start_node: str, end_nodes: List[str]):
        self.nodes = nodes
        self.start_node = start_node
        self.end_nodes = end_nodes
    
    def run(self, initial_state: Dict[str, Any], verbose: bool = True) -> Dict[str, Any]:
        """Execute the workflow."""
        state = initial_state.copy()
        current = self.start_node
        visited = []
        max_steps = 100  # Prevent infinite loops
        
        for step in range(max_steps):
            if current not in self.nodes:
                break
            
            node = self.nodes[current]
            visited.append(current)
            
            if verbose:
                print(f"  Step {step + 1}: {current}")
            
            # Execute handler
            state = node.handler(state)
            
            # Check if end node
            if current in self.end_nodes:
                break
            
            # Find next node
            next_node = None
            for condition, target in node.next_nodes:
                if condition is None or condition(state):
                    next_node = target
                    break
            
            if next_node is None:
                break
            
            current = next_node
        
        state["_workflow_trace"] = visited
        return state
    
    def visualize(self) -> str:
        """Generate text visualization of workflow."""
        lines = ["Workflow Graph:"]
        lines.append(f"  Start: {self.start_node}")
        lines.append(f"  End: {self.end_nodes}")
        lines.append("  Connections:")
        
        for name, node in self.nodes.items():
            for condition, target in node.next_nodes:
                cond_str = "(conditional)" if condition else ""
                lines.append(f"    {name} -> {target} {cond_str}")
        
        return "\n".join(lines)


# Example: Build an order processing workflow
print("=" * 60)
print("WORKFLOW BUILDER SOLUTION")
print("=" * 60)

# Define handlers
def validate_order(state):
    state["validated"] = state.get("amount", 0) > 0
    return state

def check_inventory(state):
    state["in_stock"] = state.get("quantity", 0) <= 100
    return state

def process_payment(state):
    state["payment_status"] = "completed"
    return state

def ship_order(state):
    state["shipped"] = True
    state["tracking"] = "TRK123456"
    return state

def reject_order(state):
    state["status"] = "rejected"
    return state

def complete_order(state):
    state["status"] = "completed"
    return state

# Build workflow
builder = WorkflowBuilder()

builder.add_node("validate", validate_order)
builder.add_node("check_inventory", check_inventory)
builder.add_node("payment", process_payment)
builder.add_node("ship", ship_order)
builder.add_node("reject", reject_order)
builder.add_node("complete", complete_order)

builder.set_start("validate")
builder.set_end("reject", "complete")

# Connect nodes with conditions
builder.connect("validate", "check_inventory", lambda s: s.get("validated"))
builder.connect("validate", "reject", lambda s: not s.get("validated"))
builder.connect("check_inventory", "payment", lambda s: s.get("in_stock"))
builder.connect("check_inventory", "reject", lambda s: not s.get("in_stock"))
builder.connect("payment", "ship")
builder.connect("ship", "complete")

workflow = builder.build()

# Visualize
print(workflow.visualize())

# Test cases
print("\n--- Test: Valid Order ---")
result = workflow.run({"amount": 100, "quantity": 5})
print(f"Final status: {result.get('status')}")
print(f"Trace: {result['_workflow_trace']}")

print("\n--- Test: Invalid Order ---")
result = workflow.run({"amount": 0, "quantity": 5})
print(f"Final status: {result.get('status')}")
print(f"Trace: {result['_workflow_trace']}")

---

## Key Takeaways

1. **Typed State**: TypedDict ensures type safety in workflow state
2. **Conditional Routing**: Branch workflows based on state conditions
3. **Human-in-the-Loop**: Pause for approval at critical points
4. **Parallel Execution**: Use ThreadPoolExecutor for concurrent tasks
5. **Declarative Building**: DSLs make workflows easier to understand and maintain